In [1]:
import sys
sys.path.insert(0, '../src/')
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz
import graph_statistics
import utils

from evaluation import (Evaluation, tabular_from_statistics, df_from_tabular, compute_original_statistics,
                        boxplot, df_from_dataset, errorbar_plot)

ModuleNotFoundError: No module named 'scipy'

#### True graph

In [2]:
dataset = 'rt_gop'
#dataset = 'citeseer'
_A_obs = load_npz(f'../data/datasets/{dataset}.npz')

val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.csr_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1])))
assert (train_graph.toarray() == train_graph.toarray().T).all()

#### Statistic Functions

In [4]:
statistic_fns = {'Assortativity':graph_statistics.assortativity,
                 #'Average Degree':graph_statistics.average_degree,
                 #'Claw Count':graph_statistics.claw_count,
                 'Clustering Coefficient':graph_statistics.clustering_coefficient,
                 'Characteristic Path Length':graph_statistics.compute_cpl,
#                  'Edge Distribution Entropy':graph_statistics.edge_distribution_entropy,
                 #'Gini':graph_statistics.gini,
#                   'Intra com.' : intra_community_density,
#                   'Inter com.' : inter_community_density,
                 #'LCC Size':graph_statistics.LCC,
                 'Max Degree':graph_statistics.max_degree,
                 #'Min Degree':graph_statistics.min_degree,
                 #'Num Connected Components':graph_statistics.num_connected_components,
                 'Power Law α':graph_statistics.power_law_alpha,
                 #'Spectral Gap':graph_statistics.spectral_gap,
                 'Square Count':graph_statistics.square_count,
                 'Triangle Count':graph_statistics.triangle_count,
#                  'Wedge Count':graph_statistics.wedge_count,
                 }

#### Evaluate 

In [5]:
df, evals = df_from_dataset(path_to_dataset=f'../rel_error_table/{dataset}/',
                            statistic_fns=statistic_fns,
                            target_overlap=0.54,
                            original_graph=train_graph,
                            max_trials=None)
df

,Assortativity,Clustering Coefficient,Max Degree,Power Law α,Spectral Gap,Square Count,Triangle Count,Edge Overlap (%),ROC-AUC Score,Average Precision,Time (s)
citeseer,-0.014617,0.012596,72.0,2.237941,0.001633,1866.0,483.0,1.000000,NaN,NaN,NaN
baseline_transition_ce_loss,-0.075784,0.010023,47.2,2.130685,0.012782,548.8,137.6,0.557202,0.902619,0.920768,19.380773


#### Plot at selected statistics

In [7]:
df[['Max Degree', 'Assortativity', 'Triangle Count', 'Square Count', 'Power Law α', 'Clustering Coefficient',
    'Characteristic Path Length', 'ROC-AUC Score', 'Time (s)', 'Edge Overlap (%)']]

,Max Degree,Assortativity,Triangle Count,Square Count,Power Law α,Clustering Coefficient,Spectral Gap,ROC-AUC Score,Time (s),Edge Overlap (%)
citeseer,72.0,-0.014617,483.0,1866.0,2.237941,0.012596,0.001633,NaN,NaN,1.000000
baseline_transition_ce_loss,47.2,-0.075784,137.6,548.8,2.130685,0.010023,0.012782,0.902619,19.380773,0.557202


In [6]:
df_for_paper = df.reindex(['citeseer', 'baseline_conf', 'baseline_FA', 'baseline_FT', 'baseline_FS', 
                           'baseline_FM',
                           'baseline_transition_ce_loss',
                           'baseline_Netgan_sample_many_eo_52', 
                           'Ours'])
# df_for_paper = df.reindex(['CORA-ML', 'baseline_conf', 'baseline_FS'])
df_for_paper[['Max Degree', 'Assortativity', 'Triangle Count', 'Square Count', 'Power Law α', 'Clustering Coefficient',
              'Spectral Gap', 'ROC-AUC Score', 'Time (s)', 'Edge Overlap (%)']]

NameError: name 'df_for_paper' is not defined